# Crawler 

input: 
*   the link of the project guttenburg under the category of "stories"

output: 

*   custom_dataset.txt (the file containing all the text for each book)
*   book_id.txt (a file contaning the id's from each book)
*   test_txt.txt (a file contaning a paragraph of 100 words from each book)



In [ ]:
import bs4 as bs
import urllib.request
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import drive

In [ ]:
drive.mount('/drive')

Mounted at /drive


In [ ]:
# function used to return all the text inside a url
def get_text(soup):
    for script in soup("script"):
        script.decompose()
    text = soup.get_text()
    return text

In [ ]:
#story_counter = 0
index = 1 # counter for the number of the stories we want to crawl
story_index = [] # here we save the id of stories we have already crawled

# f will be the file containing the text for all the books(each line of the txt will refer to one book)
# g will be a txt file that will contain the id's of the books we will crawl
# t will be the test file, containing a paragraph of text from each book
f = open("custom_dataset.txt", "w")
g = open("book_id.txt", "w")
t = open("test_txt.txt", "w")

# we crawled 970 total books
while index <= 970:
  if (index%50==0):
    # this is used so we know how many books we have added so far (prints once every 50 books)
    print("=========")
    print("index =", index)
  
  # link will be the link of the page Guttenberg from which we want to crawl the books
  link = "https://www.gutenberg.org/ebooks/search/?query=stories&submit_search=Go%21&start_index="+str(index)
  agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

  # we create a request to get on the links inside the initial page we mentioned above (link)
  res = requests.get(link, headers=agent)
  soup = BeautifulSoup(res.text, "html.parser")
  href = soup.find_all('a',href=True)
  

  for i in href:
    # for every link we check its length since the url of each book had a certain structure
    new_link = i['href']
    if len(new_link) > 8 and len(new_link) <= 13:
      # from each link, we extract a, which will be an integer (id of the story)
      a = list(new_link)
      a = "".join(a[8:])

      if int(a) not in story_index:
        story_index.append(int(a))

        # we create the new url by adding the story id in the end and we request to 
        # get all the links inside the above url
        first = "https://www.gutenberg.org/ebooks/"+a
        res1 = requests.get(first, headers=agent)
        soup1 = BeautifulSoup(res1.text, "html.parser")
        href1 = soup1.find_all('a',href=True)


        for j in href1:
          # for each one of the new links, we check its suffix, since the url containing 
          # the plain text of each book had a suffix of either ".txt" or ".txt.utf-8"
          j = j['href']
          if j[-4:] == ".txt" or j[-10:] == ".txt.utf-8":

            # we access the new link with the above suffixes and we request to extract all the text inside it
            second = "https://www.gutenberg.org"+j
            res2 = requests.get(second, headers=agent)
            soup2 = BeautifulSoup(res2.text, "html.parser")
            story = get_text(soup2)

            # here we check if the book we are examining is english or not by checking 
            # the plain text of the book in case it contans the phrase "Language: English"
            # we did this since not all book pages provided us with metadata about the language
            is_english = re.search(r'\b(Language: )\b', story)
            if is_english != None:
              check = is_english.end()
            else: 
              check = 10
            

            if story[check:check+7] == "English":
              # if we are examining an english story, we cut a phrase of 100 words and 
              # put it inside the test txt and we also remove annecesery text from the start
              # and from the end since every book started after the phrase:
              # "START OF THE PROJECT GUTENBERG EBOOK" and every book ended before
              # "END OF THE PROJECT GUTENBERG EBOOK"
              story = story.split()
              get_for_test = int(0.6*len(story)) 
              test_story = " ".join(story[get_for_test:get_for_test+100])
              story = " ".join(story[:get_for_test] + story[get_for_test+100:])

              #remove from the start
              text_to_be_removed2 = re.search(r'\b(START OF THE PROJECT GUTENBERG EBOOK)\b', story)
              if text_to_be_removed2 == None:
                text_to_be_removed2 = re.search(r'\b(START OF THIS PROJECT GUTENBERG EBOOK)\b', story)
              if text_to_be_removed2 != None:
                story = story[text_to_be_removed2.start() + 1000:]
              else:
                story = story[2000:]

              #remove from the end
              text_to_be_removed = re.search(r'\b(END OF THE PROJECT GUTENBERG EBOOK)\b', story)
              if text_to_be_removed == None:
                text_to_be_removed = re.search(r'\b(END OF THIS PROJECT GUTENBERG EBOOK)\b', story)
              if text_to_be_removed == None:
                text_to_be_removed = re.search(r'\b(This file should be named)\b', story)
              if text_to_be_removed != None:
                story = story[:text_to_be_removed.start()]
              else:
                story = story[:len(story) - 15000]
                
              # we update our txt files with the new informations we crawled
              t.write(test_story+"\n")
              f.write(story+"\n")
              g.write(str(a)+"\n")
            else:
              # we print the id of the book in case it was non english in order to
              # know we didnt false remove any
              print(a)
  index += 1
t.close()
f.close()
g.close()